In [ ]:
import os
import time
import datetime
import numpy as np
import pandas as pd
import scipy.stats
import matplotlib
import matplotlib.pyplot as plt
import pickle

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

fontsize = 8
lw = 0.75

matplotlib.rc('font', **{'family': 'Times New Roman', 'size': fontsize})
matplotlib.rc('axes', **{'linewidth': 0.75, 'labelsize': fontsize})
matplotlib.rc('xtick', **{'labelsize': fontsize})
matplotlib.rc('ytick', **{'labelsize': fontsize})
matplotlib.rc('xtick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.minor', **{'width': lw, 'size':1.5})

%matplotlib inline

In [ ]:
bank_holidays = {
    2019: [
        [1, 1],
        [4, 19],
        [4, 21],
        [4, 25],
        [5, 1],
        [6, 10],
        [6, 20],
        [8, 15],
        [10, 5],
        [11, 1],
        [12, 1],
        [12, 8],
        [12, 25]
    ],
    2020: [
        [1, 1],
        [4, 10],
        [4, 12],
        [4, 25],
        [5, 1],
        [6, 10],
        [6, 11],
        [8, 15],
        [10, 5],
        [11, 1],
        [12, 1],
        [12, 8],
        [12, 25]
    ]
}

In [ ]:
def mean_confidence_interval(X, confidence=0.95):
    m  = np.mean(X, axis=0)
    se = scipy.stats.sem(X, axis=0)
    h  = se * scipy.stats.t.ppf((1 + confidence) / 2., X.shape[0] - 1)
    return m, m - h, m + h

In [ ]:
def read_sheet(data_file, sheet_name, header=(0,)):
    df = pd.read_excel(data_file, sheet_name=sheet_name, parse_dates=True, header=header)
    
    if isinstance(df.columns, pd.MultiIndex):
        cols = []
        for col in df.columns.to_list():
            if 'Unnamed' in col[0]:
                cols.append(col[1])
            else:
                cols.append(col[0].split(' ')[-1] + '_' + col[1].split(' ')[0])
        df.columns = cols

    if 'date' in df.columns:
        year = df.date[0].year
        df['datetime'] = [pd.Timestamp(year=d.year, month=d.month, day=d.day, 
                                       hour=t.hour, minute=t.minute) for d,t in zip(df.date, df.time)]
        df.drop(['date', 'time'], axis=1, inplace=True)
    elif 'datetime (epoch)' in df.columns:
        datetimes = [time.gmtime(dt) for dt in df['datetime (epoch)']]
        year = datetimes[0].tm_year
        df['datetime'] = [pd.Timestamp(year = dt.tm_year, month = dt.tm_mon, day = dt.tm_mday, 
                                 hour = dt.tm_hour, minute  = dt.tm_min) for dt in datetimes]
        df.drop('datetime (epoch)', axis=1, inplace=True)

    df['weekday'] = [d.weekday() for d in df['datetime']]
    df['weekend'] = [d.weekday() in (5,6) for d in df['datetime']]
    df['holiday'] = False
    for holiday in bank_holidays[year]:
        idx = np.array([timestamp.month == holiday[0] and timestamp.day == holiday[1] \
                        for timestamp in df['datetime']])
        df.loc[idx, 'holiday'] = True

    cols = df.columns.tolist()
    cols = cols[-4:] + cols[:-4]
    df = df[cols]
    df.rename({col: col.split(' ')[0] for col in cols}, axis='columns', inplace=True)
    return df

In [ ]:
data_folder = '../data/'
xls_files = ['data1.xlsx','data2.xlsx']
pkl_files = [os.path.splitext(xls_file)[0] + '.pkl' for xls_file in xls_files]
reload = False
data = {}
for xls_file, pkl_file in zip(xls_files, pkl_files):
    if not os.path.isfile(data_folder + pkl_file) or reload:
        building_energy = read_sheet(data_folder + xls_file, 'building_energy')
        building_sensor = read_sheet(data_folder + xls_file, 'building_sensor')
        weather_data = read_sheet(data_folder + xls_file, 'weather_data')
        zones = {i+1: read_sheet(data_folder + xls_file, sheet_name=f'zone#{i+1}_energy', header=(0,1)) for i in range(5)}
        sensors = {i+1: read_sheet(data_folder + xls_file, sheet_name=f'zone#{i+1}_sensor') for i in range(3)}
        data = {'full': {
            'building_energy': building_energy,
            'building_sensor': building_sensor,
            'weather_data': weather_data,
            'zones': zones,
            'sensors': sensors
        }}
        pickle.dump(data, open(data_folder + pkl_file, 'wb'))
    blob = pickle.load(open(data_folder + pkl_file, 'rb'))
    for key1 in blob:
        if key1 not in data:
            data[key1] = blob[key1]
        else:
            for key2,value2 in blob[key1].items():
                if isinstance(value2, dict):
                    for key3,value3 in blob[key1][key2].items():
                        data[key1][key2][key3] = data[key1][key2][key3].append(value3, ignore_index=True)
                elif isinstance(value2, pd.DataFrame):
                    data[key1][key2] = data[key1][key2].append(value2, ignore_index=True)
                else:
                    raise Exception(f'Do not know how to deal with object of type {type(value2)}')
building_energy = data['full']['building_energy']
building_sensor = data['full']['building_sensor']
weather_data = data['full']['weather_data']
zones = data['full']['zones']
sensors = data['full']['sensors']

In [ ]:
t0 = datetime.datetime.combine(datetime.date.today(), building_energy['datetime'][0].to_pydatetime().time())
t1 = datetime.datetime.combine(datetime.date.today(), building_energy['datetime'][1].to_pydatetime().time())
time_step = int((t1 - t0).total_seconds() / 60) # [min]
samples_per_day = 24 * 60 // time_step
num_samples =  building_energy.shape[0]
num_days = num_samples // samples_per_day
print(f'Time step: {time_step} minutes.')
print(f'Number of days: {num_days}.')

In [ ]:
building_energy[::samples_per_day][:14]

In [ ]:
data['weekdays'] = {}
data['holidays'] = {}
for key,value in data['full'].items():
    if isinstance(value, pd.DataFrame):
        df = value
        weekdays = (df['weekday'] < 5) & (df['holiday'] == False)
        holidays = (df['weekday'] >= 5) | (df['holiday'] == True)
        data['weekdays'][key] = df.loc[weekdays, :]
        data['holidays'][key] = df.loc[holidays, :]
    elif isinstance(value, dict):
        data['weekdays'][key] = {}
        data['holidays'][key] = {}
        for subkey,df in value.items():
            weekdays = (df['weekday'] < 5) & (df['holiday'] == False)
            holidays = (df['weekday'] >= 5) | (df['holiday'] == True)
            data['weekdays'][key][subkey] = df.loc[weekdays, :]
            data['holidays'][key][subkey] = df.loc[holidays, :]

In [ ]:
building_consumption = {}
building_generation = {}
building_temperature = {}
for key,value in data.items():
    tmp = value['building_energy']['consumption'].to_numpy()
    building_consumption[key] = np.reshape(tmp, [tmp.size // samples_per_day, samples_per_day], order='C')
    tmp = value['building_energy']['generation'].to_numpy()
    building_generation[key] = np.reshape(tmp, [tmp.size // samples_per_day, samples_per_day], order='C')
    tmp = value['building_sensor']['temperature'].to_numpy()
    building_temperature[key] = np.reshape(tmp, [tmp.size // samples_per_day, samples_per_day], order='C')

In [ ]:
t = np.arange(samples_per_day) * time_step / 60
fig,ax = plt.subplots(3, 1, figsize=(8.5 / 2.5, 3 * 2.5 / 2.54), sharex=True)
cmap = {'full': [.4,.4,.4], 'weekdays': [.2,.6,.2], 'holidays': [.7,.2,.7]}
lgnd = {k: k.capitalize() for k in cmap}
lgnd['full'] = 'Global'
offset = .5 + np.zeros(3)
for key,value in building_consumption.items():
    m, lower, upper = mean_confidence_interval(value)
    m, lower, upper = m*1e-3, lower*1e-3, upper*1e-3
    col = np.min([[1,1,1], cmap[key] + np.array(offset)], axis=0)
    ax[0].fill_between(t, lower, upper, color=col)
    ax[0].plot(t, m, color=cmap[key], lw=1, label=lgnd[key])
for key,value in building_generation.items():
    m, lower, upper = mean_confidence_interval(value)
    m, lower, upper = m*1e-3, lower*1e-3, upper*1e-3
    col = np.min([[1,1,1], cmap[key] + np.array(offset)], axis=0)
    ax[1].fill_between(t, lower, upper, color=col)
    ax[1].plot(t, m, color=cmap[key], lw=1, label=lgnd[key])
for key,value in building_temperature.items():
    m, lower, upper = mean_confidence_interval(value)
    col = np.min([[1,1,1], cmap[key] + np.array(offset)], axis=0)
    ax[2].fill_between(t, lower, upper, color=col)
    ax[2].plot(t, value.mean(axis=0), color=cmap[key], lw=1, label=lgnd[key])
for a in ax:
    a.grid(True, which='major', axis='y', color=[.8,.8,.8], linestyle='-', linewidth=0.5)
    for side in 'top','right':
        a.spines[side].set_visible(False)
ax[-1].set_xlabel('Time of the day [hour]')

ax[0].set_yticks(np.r_[3000 : 6100 : 1000] * 1e-3)
ax[0].set_ylabel('Consump. [kW]')

ax[1].set_yticks(np.r_[0 : 3100 : 1000] * 1e-3)
ax[1].set_ylabel('Gener. [kW]')

ax[2].set_ylabel('Temp. [C]')
ax[2].set_xticks(np.r_[0 : 25 : 2])
ax[2].set_yticks(np.r_[14 : 21 : 2])
ax[0].legend(loc='upper left', frameon=False, fontsize=fontsize)
fig.tight_layout(pad=0.5)
for suffix in '.pdf','.png':
    fig.savefig('building_data_mean' + suffix, dpi=600)

In [ ]:
fig,ax = plt.subplots(2, 1)
x = building_energy['consumption']
y = building_energy['generation']
ax[0].plot(x, y, 'k.', markersize=2)
H,xedges,yedges = np.histogram2d(x, y, bins=101)
X,Y = np.meshgrid(yedges[:-1], xedges[:-1])
ax[1].contourf(Y, X, np.log10(H))

In [ ]:
fig,ax = plt.subplots(2, 1)
x = building_sensor['temperature']
y = building_energy['generation']
ax[0].plot(x, y, 'k.', markersize=2)
H,xedges,yedges = np.histogram2d(x, y, bins=101)
X,Y = np.meshgrid(yedges[:-1], xedges[:-1])
ax[1].contourf(Y, X, np.log10(H))

In [ ]:
lw = 0.5
col = 'k'
fig = plt.figure(figsize=(8.5 / 2.54, 4))

black = [0, 0, 0]
red = [.7, 0, 0]
green = [0, .5, 0]

rows,cols = 3,1
offset = [[0.13, 0.02], [0.35,0.025]]
space = [0.1, 0.03]
w = (1 - np.sum(offset[0]) - (cols-1) * space[0]) / cols
h = (1 - np.sum(offset[1]) - (rows-1) * space[1]) / rows
ax = [
    plt.axes([
        offset[0][0],
        offset[1][0] + i * (h + space[1]),
        w,
        h
    ]) for i in range(rows-1, -1, -1)
]

rows,cols = 1,3
offset[0][0] = 0.07
offset[1] = [0.018, 0.8]
space = [0.07, 0.03]
w = (1 - np.sum(offset[0]) - (cols-1) * space[0]) / cols
h = (1 - np.sum(offset[1]) - (rows-1) * space[1]) / rows
small_ax = [
    plt.axes([offset[0][0] + i * (w + space[0]), offset[1][0], w, h])
    for i in range(cols)
]

df = building_energy.copy()
df['consumption'] *= 1e-3
df['generation'] *= 1e-3
df.plot(x='datetime', y='consumption', color=black,
                     lw=lw, ax=ax[0], legend=False, xlabel='')
df.plot(x='datetime', y='generation', color=red,
                     lw=lw, ax=ax[1], legend=False, xlabel='')
building_sensor.plot(x='datetime', y='temperature', color=green,
                     lw=lw, ax=ax[2], legend=False, xlabel='')

start = pd.Timestamp(year=2019, month=1, day=1)
stop = pd.Timestamp(year=2019, month=12, day=31)

ylim = [[1, 14], [-0.2, 6.1], [0,34]]
yticks = [np.r_[2 : 15 : 3], np.r_[0 : 6.1 : 2], np.r_[0 : 36 : 10]]
ylabels = ['Consum. [kW]', 'Gener. [kW]', 'Temp. [C]']
for a,yl,yt,lbl in zip(ax, ylim, yticks, ylabels):
    if a != ax[-1]:
        a.set_xticklabels([])
    for side in 'right','top':
        a.spines[side].set_visible(False)
    a.grid(which='major', axis='y', color=[.6,.6,.6], lw=0.5, linestyle=':')
    a.set_xlim([start, stop])
    a.set_ylim(yl)
    a.set_yticks(yt)
    a.set_ylabel(lbl)


start = pd.Timestamp(year=2019, month=3, day=11)
stop = pd.Timestamp(year=2019, month=3, day=13)
idx = (df.datetime >= start) & (df.datetime < stop)
df.loc[idx,:].plot(x='datetime', y='consumption', color=black,
                     lw=lw, ax=small_ax[0], legend=False, xlabel='')
df.loc[idx,:].plot(x='datetime', y='generation', color=red,
                     lw=lw, ax=small_ax[1], legend=False, xlabel='')
idx = (building_sensor.datetime >= start) & (building_sensor.datetime < stop)
building_sensor.loc[idx,:].plot(x='datetime', y='temperature', color=green,
                     lw=lw, ax=small_ax[2], legend=False, xlabel='')

ylim = [[2, 10], [-0.2, 6.1], [9,23]]
yticks = [np.r_[2 : 11 : 2], np.r_[0 : 6.1 : 2], np.r_[10 : 23 : 4]]
ylabels = ['Consumption', 'Generation', 'Temperature']
for a,yl,yt,lbl in zip(small_ax, ylim, yticks, ylabels):
    for side in 'right','top':
        a.spines[side].set_visible(False)
    a.grid(which='major', axis='y', color=[.6,.6,.6], lw=0.5, linestyle=':')
    a.set_xticks([start, stop])
    a.set_ylim(yl)
    a.set_yticks(yt)
    a.set_title(lbl, fontsize=fontsize)

# fig.tight_layout()
fig.savefig('building_data.pdf')
# for suffix in '.pdf','.png':
#     fig.savefig('building_data_full' + suffix, dpi=600)
#     fig.savefig('building_data_jan' + suffix, dpi=600)
#     fig.savefig('building_data_jan_14-21' + suffix, dpi=600)

In [ ]:
fig = plt.figure()
ax = plt.axes([0.1, 0.1, 0.8, 0.8])
ax.plot(np.linspace(0, np.pi, 100), np.sin(np.linspace(0, np.pi, 100)))
ax.set_xlabel('x')
ax.set_ylabel('y')
for child in ax.get_children():
    print(child)
    if isinstance(child, matplotlib.text.Text):
        print('Text: "' + child.get_text() + '"')
        child.set_visible(False)

In [ ]:
N_zones = len(zones)
fig,ax = plt.subplots(N_zones, 2, figsize=(12, 2 * N_zones), sharex=True)
for a,(key,zone) in zip(ax[:,0], zones.items()):
    N_HVAC = np.sum(['HVAC' in col for col in zone.columns]) // 3
    if N_HVAC > 1:
        zone.plot(x='datetime', y=[f'HVAC#{i+1}_power' for i in range(N_HVAC)], lw=0.5, ax=a)
    else:
        zone.plot(x='datetime', y='HVAC_power', lw=0.5, ax=a)
    for side in 'right','top':
        a.spines[side].set_visible(False)
    a.set_title(f'Zone {key}')
    a.set_ylabel('Consumption [W]')
a.set_xlabel('Date')
for a,(key,zone) in zip(ax[:,1], zones.items()):
    zone.plot(x='datetime', y=['light_power','sockets_power'], lw=0.5, ax=a)
    for side in 'right','top':
        a.spines[side].set_visible(False)
    a.set_title(f'Zone {key}')
a.set_xlabel('Date')
if 'data1' in xls_file:
    start = pd.Timestamp(year=2019, month=1, day=1)
    stop = pd.Timestamp(year=2019, month=12, day=31)
    ax[0,0].set_xlim([start, stop])
fig.tight_layout()
for suffix in '.pdf','.png':
    if suffix == '.pdf':
        continue
    fig.savefig('building_zones_data' + suffix, dpi=600)

In [ ]:
fig,ax = plt.subplots(3, 1, figsize=(8, 2 * 3), sharex=True)
window_size = 3 * 60
columns = 'temperature', 'humidity', 'radiation'
units = 'C', '%', 'Wm^2'
for i,(column,unit) in enumerate(zip(columns,units)):
    ax[i].plot(weather_data['datetime'],
               weather_data[column].rolling(window_size // time_step, win_type='triang').mean(),
               'k', lw=0.5)
    ax[i].set_ylabel(column.capitalize() + f' [{unit}]')
    for side in 'top','right':
        ax[i].spines[side].set_visible(False)
ax[-1].set_xlabel('Date')
if 'data1' in xls_file or True:
    start = pd.Timestamp(year=2019, month=1, day=1)
    stop = pd.Timestamp(year=2019, month=1, day=31)
    #ax[-1].set_xlim([start, stop])
fig.tight_layout()
for suffix in '.pdf','.png':
    fig.savefig('weather_data_full' + suffix, dpi=600)
    #fig.savefig('weather_data_jan_14-21' + suffix, dpi=600)
    #fig.savefig('weather_data_jan' + suffix, dpi=600)